# Recurrent Neural Network

In [1]:
import keras
import numpy as np
import random
import matplotlib.pyplot as plt

Using TensorFlow backend.


# Getting our Data

In [4]:
import io
path = keras.utils.data_utils.get_file('nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
with io.open(path, encoding='UTF-8') as f:
    text = f.read().lower()
print(len(text))

606208/600901 [==============================] - 0s 1us/step
600893


In [5]:
chars = sorted(list(set(text)))
print(len(chars))

57


In [7]:
char_indices = dict ((c,i) for i,c in enumerate(chars))
indices_char = dict ((i,c) for i,c in enumerate(chars))

In [8]:
sentences = []
next_chars = []

for i in range(0, len(text)-40,3):
    sentences.append(text[i:i+40])
    next_chars.append(text[i+40])

x = np.zeros((len(sentences),40, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

for i, sentence in enumerate(sentences):
    for t,char in enumerate(sentence):
        x[i,t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] =1
    

# Building Model

In [9]:
model = keras.models.Sequential()
model.add(keras.layers.LSTM(128, input_shape=(40, len(chars))))
model.add(keras.layers.Dense(len(chars), activation='softmax'))

# Compiling the model

In [10]:
model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.RMSprop(lr=0.01))

# Lets create function that gives us some sample text

In [11]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float')
    preds = np.log(preds)/temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

# Fitting our Model

In [13]:
model.fit(x, y, batch_size=18,epochs=1)
start_index = random.randint(0, len(text)-40-1)
for diversity in [0.2,0.5,1.0,1.2]:
    generated = ''
    sentence = text[start_index: start_index+40]
    generated +=sentence
    
    for i in range(600):
        x_pred = np.zeros((1,40,len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0,t,char_indices[char]] = 1
        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]
        
        generated += next_char
        sentence = sentence[1:] + next_char
        
    with open('Gen.txt','w') as f:
        f.write(generated)

Epoch 1/1
200285/200285 [==============================] - 826s 4ms/step - loss: 1.9838
